In [4]:
%load_ext autoreload
%autoreload 2

import h5py
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
import util as yu

path='dat/NST_b_cD96.h5'
with h5py.File(path) as f:
    dat=(f['diags/N/data/N2_N2'][:,:,0]+f['diags/N/data/N2_N2'][:,:,1])/2
    C2pt_dat=np.real(dat)
    
aLat=0.05692 # lattice spacing a in fm
aInvLat=1/aLat*197.3/1000 # a^{-1} in GeV


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
settings={
    'fitmins_1st':range(8,34+1),
    'fitmins_2st':range(1,20+1),
    'fitmins_3st':range(1,7+1),
    'ylim1':[0.6,1.7],
    'ylim2':[0.9,1.04],
    'ylim3':[0.9,3.0],
}

chi2Size=9
xUnit=aLat; yUnit=aInvLat

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))

def run(corrQ=True):
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    fig.suptitle('Correlated fit to C2pt' if corrQ else 'Uncorrelated fit to C2pt')
    
    ax1.set_xlabel(r'$t$ [fm]')
    ax2.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax3.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax1.set_ylabel(r'$m_N^{\mathrm{eff}}$ [GeV]')
    ax2.set_ylabel(r'$m_N$ [GeV]')
    ax3.set_ylabel(r'$E_1$ [GeV]')
    ax1.set_ylim(settings['ylim1'])
    ax2.set_ylim(settings['ylim2'])
    ax3.set_ylim(settings['ylim3'])
    
    mN_exp=0.938
    ax1.axhline(y=mN_exp,color='black',linestyle = '--', marker='')
    ax2.axhline(y=mN_exp,color='black',linestyle = '--', marker='', label=r'$m_N^{\mathrm{exp}}=$'+'%0.3f'%mN_exp)
    C2pt_jk=yu.jackknife(C2pt_dat)
    C2pt_mean,C2pt_err=yu.jackme(C2pt_jk)
    C2pt_rela=np.abs(C2pt_err/C2pt_mean)
    temp=[(i,rela) for i,rela in enumerate(C2pt_rela) if rela>0.2]
    fitmax=temp[0][0]-1
    
    func=lambda C2pt: np.log(C2pt/np.roll(C2pt,-1,axis=0))
    mEff_jk=yu.jackmap(func,C2pt_jk)
    (mEff_mean,mEff_err)=yu.jackme(mEff_jk)
    tmin=1; tmax=fitmax+1
    plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mEff_mean[tmin:tmax]*yUnit; plt_err=mEff_err[tmin:tmax]*yUnit
    ax1.errorbar(plt_x,plt_y,plt_err,color='black',fmt='s',mfc='white')

    pars0_initial=[0.4,1e-8,0.5,2,0.8,1]
    
    # 1st fits
    color='r'
    fitmins=settings['fitmins_1st']
    pars0=pars0_initial[:2]; Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_C2pt_1st(tList,*pars)
        def estimator(pars):
            return [pars[0]]
        y_jk=C2pt_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    pars0=obs_mean_MA[-Npar:]
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2,label=r'$m_N^{\mathrm{1st}}=$'+yu.un2str(plt_y,plt_err))
    
    # 2st fits
    color='g'
    fitmins=settings['fitmins_2st']
    pars0=np.hstack([pars0,pars0_initial[2:4]]); Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_C2pt_2st(tList,*pars)
        def estimator(pars):
            return [pars[0],pars[0]+pars[2]]
        y_jk=C2pt_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[1]*yUnit; plt_err=obs_err[1]*yUnit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    pars0=obs_mean_MA[-Npar:]
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{2st}}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[1]*yUnit; plt_err=obs_err_MA[1]*yUnit
    ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2)
    
    
    # 3st fits
    color='b'
    fitmins=settings['fitmins_3st']
    pars0=np.hstack([pars0,pars0_initial[4:6]]); Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_C2pt_3st(tList,*pars)
        def estimator(pars):
            return [pars[0],pars[0]+pars[2]]
        y_jk=C2pt_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[1]*yUnit; plt_err=obs_err[1]*yUnit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_err=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{3st}}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[1]*yUnit; plt_err=obs_err_MA[1]*yUnit
    ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2)
    
    ax2.legend(loc=(0.6,0.5),fontsize=12)
    
    plt.tight_layout()
    plt.savefig('fig/fit_2pt_cor.pdf' if corrQ else 'fig/fit_2pt_unc.pdf')
    plt.close()
    
run(True)
run(False)

In [6]:
settings={
    'fitmins_1st':range(8,34+1),
    'fitmins_2st':range(1,20+1),
    'fitmins_3st':range(1,7+1),
    'ylim1':[0.6,1.7],
    'ylim2':[0.9,1.04],
    'ylim3':[0.9,3.0],
}

chi2Size=9
xUnit=aLat; yUnit=aInvLat

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))
func_mEff_1st=lambda t,E0: np.log(func_C2pt_1st(t,E0,1)/func_C2pt_1st(t+1,E0,1))
func_mEff_2st=lambda t,E0,dE1,rc1: np.log(func_C2pt_2st(t,E0,1,dE1,rc1)/func_C2pt_2st(t+1,E0,1,dE1,rc1))
func_mEff_3st=lambda t,E0,dE1,rc1,dE2,rc2: np.log(func_C2pt_3st(t,E0,1,dE1,rc1,dE2,rc2)/func_C2pt_3st(t+1,E0,1,dE1,rc1,dE2,rc2))


def run(corrQ=True):
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    fig.suptitle('Correlated fit to mEff' if corrQ else 'Uncorrelated fit to mEff')
    
    ax1.set_xlabel(r'$t$ [fm]')
    ax2.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax3.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax1.set_ylabel(r'$m_N^{\mathrm{eff}}$ [GeV]')
    ax2.set_ylabel(r'$m_N$ [GeV]')
    ax3.set_ylabel(r'$E_1$ [GeV]')
    ax1.set_ylim(settings['ylim1'])
    ax2.set_ylim(settings['ylim2'])
    ax3.set_ylim(settings['ylim3'])
    
    mN_exp=0.938
    ax1.axhline(y=mN_exp,color='black',linestyle = '--', marker='')
    ax2.axhline(y=mN_exp,color='black',linestyle = '--', marker='', label=r'$m_N^{\mathrm{exp}}=$'+'%0.3f'%mN_exp)
    C2pt_jk=yu.jackknife(C2pt_dat)
    func=lambda C2pt: np.log(C2pt/np.roll(C2pt,-1,axis=0))
    mEff_jk=yu.jackmap(func,C2pt_jk)
    (mEff_mean,mEff_err)=yu.jackme(mEff_jk)
    mEff_rela=np.abs(mEff_err/mEff_mean)
    temp=[(i,rela) for i,rela in enumerate(mEff_rela) if rela>0.2]
    fitmax=temp[0][0]-1

    tmin=1; tmax=fitmax+1
    plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mEff_mean[tmin:tmax]*yUnit; plt_error=mEff_err[tmin:tmax]*yUnit
    ax1.errorbar(plt_x,plt_y,plt_error,color='black',fmt='s',mfc='white')

    pars0_initial=[0.4,0.5,2,0.8,1]
    
    # 1st fits
    color='r'
    fitmins=settings['fitmins_1st']
    pars0=pars0_initial[:1]; Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_mEff_1st(tList,*pars)
        def estimator(pars):
            return [pars[0]]
        y_jk=mEff_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    pars0=obs_mean_MA[-Npar:]
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_error=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2,label=r'$m_N^{\mathrm{1st}}=$'+yu.un2str(plt_y,plt_error))
    
    # 2st fits
    color='g'
    fitmins=settings['fitmins_2st']
    pars0=np.hstack([pars0,pars0_initial[1:3]]); Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_mEff_2st(tList,*pars)
        def estimator(pars):
            return [pars[0],pars[0]+pars[1]]
        y_jk=mEff_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[1]*yUnit; plt_err=obs_err[1]*yUnit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    pars0=obs_mean_MA[-Npar:]
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_error=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2, label=r'$m_N^{\mathrm{2st}}=$'+yu.un2str(plt_y,plt_error))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[1]*yUnit; plt_error=obs_err_MA[1]*yUnit
    ax3.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2)
    
    
    # 3st fits
    color='b'
    fitmins=settings['fitmins_3st']
    pars0=np.hstack([pars0,pars0_initial[3:5]]); Npar=len(pars0)
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            return func_mEff_3st(tList,*pars)
        def estimator(pars):
            return [pars[0],pars[0]+pars[1]]
        y_jk=mEff_jk[:,tList]
        obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0,estimator=estimator,mask=None if corrQ else 'uncorrelated')
        (obs_mean,obs_err)=yu.jackme(obs_jk)
        pars0=obs_mean[-Npar:]
        fits.append([obs_mean,obs_err,chi2R,Ndof])
         
        plt_x=fitmin*xUnit; plt_y=obs_mean[0]*yUnit; plt_err=obs_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xUnit; plt_y=obs_mean[1]*yUnit; plt_err=obs_err[1]*yUnit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white')
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
    obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[0]*yUnit; plt_error=obs_err_MA[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2, label=r'$m_N^{\mathrm{3st}}=$'+yu.un2str(plt_y,plt_error))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=obs_mean_MA[1]*yUnit; plt_error=obs_err_MA[1]*yUnit
    ax3.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2)
    
    ax2.legend(loc=(0.6,0.5),fontsize=12)
    
    plt.tight_layout()
    plt.savefig('fig/fit_mEff_cor.pdf' if corrQ else 'fig/fit_mEff_unc.pdf')
    plt.close()
    
run(True)
run(False)